In [1]:
import os
import contextlib

import pandas as pd
import seaborn as sns
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def find_sheet_name(sheet_names):
    cpi_sheet_names = list(filter(lambda x: "fdi" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    cpi_sheet_names = list(filter(lambda x: "dtnn" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    if len(sheet_names) == 43:
        return "22"

    cpi_sheet_names = list(filter(lambda x: "sheet1" in x.lower(), sheet_names))

    if len(cpi_sheet_names) == 1:
        return cpi_sheet_names[0]

    raise RuntimeError(sheet_names, len(sheet_names))

In [3]:
en_to_vn = {
    "TOTAL": "TỔNG SỐ",
    "By province": "Phân theo một số địa phương",
    "Bac Lieu": "Bạc Liêu",
    "Hai Phong": "Hải Phòng",
    "Binh Duong": "Bình Dương",
    "Ha Nam": "Hà Nam",
    "Ha Noi": "Hà Nội",
    "Ho Chi Minh city": "TP. Hồ Chí Minh",
    "Tay Ninh": "Tây Ninh",
    "Bac Ninh": "Bắc Ninh",
    "Quang Ninh": "Quảng Ninh",
    "Bac Giang": "Bác Giang",
    "Quang Binh": "Quảng Bình",
    "Ba Ria - Vung Tau": "Bà Rịa - Vũng Tàu",
    "Long An": "Long An",
    "Thanh Hoa": "Thanh Hóa",
    "Dong Nai": "Đồng Nai",
    "Phu Tho": "Phú Thọ",
    "Da Nang": "Đà Nẵng",
    "Hai Duong": "Hải Dương",
    "Vinh Phuc": "Vĩnh Phúc",
    "Vinh Long": "Vĩnh Long",
    "By country and geographical territory": "Phân theo một số nước và vùng lãnh thổ",
    "Singapore": "Xin-ga-po",
    "China": "Trung Quốc",
    "South Korea": "Hàn Quốc",
    "Special Administration Hong Kong": "Đặc khu HC Hồng Công (TQ)",
    "Taiwan": "Đài Loan",
    "Japan": "Nhật Bản",
    "Philippines": "Phi-li-pin",
    "BritishVirginIslands": "Quần đảo Virgin thuộc Anh",
    "Thailand": "Thái Lan",
    "Netherlands": "Hà Lan",
    "Cayman Islands": "Quần đảo Cay-man",
    "Samoa": "Xa-moa",
    "United State": "Hoa Kỳ",
    "Seychelles": "Xây-Sen",
    "Malaysia": "Ma-lai-xi-a",
    "France": "Pháp",
    "Germany": "Đức",
}

In [4]:
from datetime import timedelta


def get_cpi(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "Phân theo một số địa phương")
                | (df[df.columns[1]] == "By province")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    if (
        len(
            df[df.columns[1]][
                (df[df.columns[1]] == "TP. Hồ Chí Minh")
                | (df[df.columns[1]] == "Ho Chi Minh city")
                | (df[df.columns[1]] == " TP. Hồ Chí Minh ")
            ].index
        )
        > 0
    ):
        df[df.columns[1]] = df[df.columns[1]].fillna(df[df.columns[0]])
        df = df.drop(columns=[df.columns[0]])

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
        .str.strip()
    )

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "TỔNG SỐ") | ((df[df.columns[0]] == "TOTAL"))
                ].index[0]
            )
        )
    )

    if len(df[df.columns[0]][df[df.columns[0]] == "TOTAL"].index) > 0:
        df[df.columns[0]] = df[df.columns[0]].map(en_to_vn)

    df = df.dropna(axis="columns", how="all")
    df = df.dropna(axis="index", how="all")

    if len(df.columns) == 4:
        df.columns = [
            "Phân Loại",
            "Số dự án",
            "Vốn đăng ký cấp mới",
            "Vốn đăng ký điều chỉnh",
        ]

        df = df[pd.to_numeric(df["Số dự án"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["Vốn đăng ký cấp mới"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["Vốn đăng ký điều chỉnh"], errors="coerce").notnull()]

        df["Vốn đăng ký"] = df["Vốn đăng ký cấp mới"] + df["Vốn đăng ký điều chỉnh"]
    elif len(df.columns) == 3:
        df.columns = [
            "Phân Loại",
            "Số dự án",
            "Vốn đăng ký",
        ]

        df = df[pd.to_numeric(df["Số dự án"], errors="coerce").notnull()]
        df = df[pd.to_numeric(df["Vốn đăng ký"], errors="coerce").notnull()]
    else:
        raise RuntimeError

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    df["Month"] = df["Date"].dt.strftime("%Y-%m")
    df["PreMonth"] = (df["Date"] - timedelta(days=31)).dt.strftime("%Y-%m")

    df = df[~df["Phân Loại"].isnull()]

    return df


xl = pd.ExcelFile("../datas/2024-03-29-02-Bieu-3.2024-ngay-28.3.xlsx")

sheet_name = find_sheet_name(xl.sheet_names)

get_cpi(xl, sheet_name, "2024-03-29")['Phân Loại'].unique().tolist()

['TỔNG SỐ',
 'Hà Nội',
 'Quảng Ninh',
 'Bắc Ninh',
 'Thái Nguyên',
 'Hưng Yên',
 'Bà Rịa - Vũng Tàu',
 'Đồng Nai',
 'Hải Phòng',
 'Long An',
 'Bình Dương',
 'TP. Hồ Chí Minh',
 'Bắc Giang',
 'Thái Bình',
 'Nam Định',
 'Hà Nam',
 'Vĩnh Phúc',
 'Nghệ An',
 'Xin-ga-po',
 'Đặc khu hành chính Hồng Kông (TQ)',
 'Trung Quốc',
 'Nhật Bản',
 'Hàn Quốc',
 'Đài Loan',
 'Vương quốc Anh',
 'Xa-moa',
 'I-ta-li-a',
 'Thái Lan',
 'Xây-xen',
 'Hoa Kỳ',
 'Ma-lai-xi-a',
 'Quần đảo Vigin thuộc Anh']

In [5]:
import os
import contextlib

dfs_fdi_origin = []
for file_name in sorted(
    os.listdir("../datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("../datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2016:
            continue

        xl = pd.ExcelFile(file_path)

        df_fdi = get_cpi(xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}")

        dfs_fdi_origin.append(df_fdi)
    except Exception as e:
        print(file_name)
        raise e

In [29]:
dfs_fdi = pd.concat(dfs_fdi_origin, ignore_index=True, axis=0)
dfs_fdi["Date"] = pd.to_datetime(dfs_fdi["Date"], dayfirst=False)
dfs_fdi

,Phân Loại,Số dự án,Vốn đăng ký cấp mới,Vốn đăng ký điều chỉnh,Vốn đăng ký,Date,Month,PreMonth
0,TỔNG SỐ,966,7110.212866,1234.628932,8344.841798,2024-04-29,2024-04,2024-03
1,Bà Rịa - Vũng Tàu,17,1521.095655,-9.557457,1511.538198,2024-04-29,2024-04,2024-03
2,Hà Nội,73,1008.129751,78.998896,1087.128647,2024-04-29,2024-04,2024-03
3,Quảng Ninh,15,737.965228,-4.507555,733.457673,2024-04-29,2024-04,2024-03
4,Bắc Ninh,142,595.713546,369.239817,964.953363,2024-04-29,2024-04,2024-03
...,...,...,...,...,...,...,...,...
3569,Quần đảo Vigin thuộc Anh,3,NaN,NaN,9.4,2016-01-29,2016-01,2015-12
3570,Đài Loan,6,NaN,NaN,8.7,2016-01-29,2016-01,2015-12
3571,Thái Lan,3,NaN,NaN,6.3,2016-01-29,2016-01,2015-12
3572,Hà Lan,1,NaN,NaN,6,2016-01-29,2016-01,2015-12


In [30]:
merge_df = pd.merge(
    left=dfs_fdi,
    right=dfs_fdi,
    left_on=["Phân Loại", "PreMonth"],
    right_on=["Phân Loại", "Month"],
    how="left",
)
dfs_fdi["Số dự án trong tháng"] = merge_df["Số dự án_x"] - merge_df["Số dự án_y"]
dfs_fdi.loc[dfs_fdi["Số dự án trong tháng"] < 0, "Số dự án trong tháng"] = merge_df[
    "Số dự án_x"
]

dfs_fdi["Vốn đăng ký cấp mới trong tháng"] = (
    merge_df["Vốn đăng ký cấp mới_x"] - merge_df["Vốn đăng ký cấp mới_y"]
)
dfs_fdi.loc[
    dfs_fdi["Vốn đăng ký cấp mới trong tháng"] < 0, "Vốn đăng ký cấp mới trong tháng"
] = merge_df["Vốn đăng ký cấp mới_x"]

dfs_fdi["Vốn đăng ký điều chỉnh trong tháng"] = (
    merge_df["Vốn đăng ký điều chỉnh_x"] - merge_df["Vốn đăng ký điều chỉnh_y"]
)
dfs_fdi.loc[
    dfs_fdi["Vốn đăng ký điều chỉnh trong tháng"] < 0,
    "Vốn đăng ký điều chỉnh trong tháng",
] = merge_df["Vốn đăng ký điều chỉnh_x"]

dfs_fdi["Vốn đăng ký trong tháng"] = (
    merge_df["Vốn đăng ký_x"] - merge_df["Vốn đăng ký_y"]
)
dfs_fdi.loc[dfs_fdi["Vốn đăng ký trong tháng"] < 0, "Vốn đăng ký trong tháng"] = (
    merge_df["Vốn đăng ký_x"]
)

dfs_fdi

,Phân Loại,Số dự án,Vốn đăng ký cấp mới,Vốn đăng ký điều chỉnh,Vốn đăng ký,Date,Month,PreMonth,Số dự án trong tháng,Vốn đăng ký cấp mới trong tháng,Vốn đăng ký điều chỉnh trong tháng,Vốn đăng ký trong tháng
0,TỔNG SỐ,966,7110.212866,1234.628932,8344.841798,2024-04-29,2024-04,2024-03,322,2336.203461,300.029434,2636.232895
1,Bà Rịa - Vũng Tàu,17,1521.095655,-9.557457,1511.538198,2024-04-29,2024-04,2024-03,12,1225.679913,-9.557457,1174.716009
2,Hà Nội,73,1008.129751,78.998896,1087.128647,2024-04-29,2024-04,2024-03,26,103.883749,57.432063,161.315812
3,Quảng Ninh,15,737.965228,-4.507555,733.457673,2024-04-29,2024-04,2024-03,5,163.229628,2.705882,165.93551
4,Bắc Ninh,142,595.713546,369.239817,964.953363,2024-04-29,2024-04,2024-03,54,114.614023,129.485,244.099023
...,...,...,...,...,...,...,...,...,...,...,...,...
3569,Quần đảo Vigin thuộc Anh,3,NaN,NaN,9.4,2016-01-29,2016-01,2015-12,NaN,NaN,NaN,NaN
3570,Đài Loan,6,NaN,NaN,8.7,2016-01-29,2016-01,2015-12,NaN,NaN,NaN,NaN
3571,Thái Lan,3,NaN,NaN,6.3,2016-01-29,2016-01,2015-12,NaN,NaN,NaN,NaN
3572,Hà Lan,1,NaN,NaN,6,2016-01-29,2016-01,2015-12,NaN,NaN,NaN,NaN


In [31]:
convert_pl = {
    "TỔNG SỐ": "TỔNG SỐ",
    "Hà Nội": "Hà Nội",
    "Quảng Ninh": "Quảng Ninh",
    "Bắc Ninh": "Bắc Ninh",
    "Thái Nguyên": "Thái Nguyên",
    "Hưng Yên": "Hưng Yên",
    "Bà Rịa - Vũng Tàu": "Bà Rịa - Vũng Tàu",
    "Đồng Nai": "Đồng Nai",
    "Hải Phòng": "Hải Phòng",
    "Long An": "Long An",
    "Bình Dương": "Bình Dương",
    "TP. Hồ Chí Minh": "TP. Hồ Chí Minh",
    "Bắc Giang": "Bắc Giang",
    "Thái Bình": "Thái Bình",
    "Nam Định": "Nam Định",
    "Hà Nam": "Hà Nam",
    "Vĩnh Phúc": "Vĩnh Phúc",
    "Nghệ An": "Nghệ An",
    "Xin-ga-po": "Xin-ga-po",
    "Đặc khu hành chính Hồng Kông (TQ)": "Đặc khu hành chính Hồng Kông (TQ)",
    "Trung Quốc": "Trung Quốc",
    "Nhật Bản": "Nhật Bản",
    "Hàn Quốc": "Hàn Quốc",
    "Đài Loan": "Đài Loan",
    "Vương quốc Anh": "Vương quốc Anh",
    "Xa-moa": "Xa-moa",
    "I-ta-li-a": "I-ta-li-a",
    "Thái Lan": "Thái Lan",
    "Xây-xen": "Xây-Sen",
    "Hoa Kỳ": "Hoa Kỳ",
    "Ma-lai-xi-a": "Ma-lai-xi-a",
    "Quần đảo Vigin thuộc Anh": "Quần đảo Vigin thuộc Anh",
    "Thanh Hóa": "Thanh Hóa",
    "Thừa Thiên Huế": "Thừa Thiên - Huế",
    "Ninh Bình": "Ninh Bình",
    "Đà Nẵng": "Đà Nẵng",
    "Ca-na-da": "Ca-na-da",
    "Pháp": "Pháp",
    "Phần Lan": "Phần Lan",
    "Na-uy": "Na-uy",
    "Yên Bái": "Yên Bái",
    "Hải Dương": "Hải Dương",
    "Phú Thọ": "Phú Thọ",
    "Bình Phước": "Bình Phước",
    "Đặc khu HC Hồng Công (TQ)": "Đặc khu hành chính Hồng Kông (TQ)",
    "Đức": "Đức",
    "Hà Lan": "Hà Lan",
    "Xa-Moa": "Xa-moa",
    "Thụy Điển": "Thụy Điển",
    "Xây-Sen": "Xây-Sen",
    "Đan Mạch": "Đan Mạch",
    "Ấn Độ": "Ấn Độ",
    "Quần đảo Virgin thuộc Anh": "Quần đảo Vigin thuộc Anh",
    "I-xa-ren": "I-xa-ren",
    "Tây Ninh": "Tây Ninh",
    "Quảng Ngãi": "Quảng Ngãi",
    "I-xa_ren": "I-xa-ren",
    "Đặc khu hành chính Hồng Công (Trung Quốc)": "Đặc khu hành chính Hồng Kông (TQ)",
    "Xây- Sen": "Xây-Sen",
    "Sóc Trăng": "Sóc Trăng",
    "Đăk Lăk": "Đăk Lăk",
    "Đặc khu hành chính Hồng Công (TQ)": "Đặc khu hành chính Hồng Kông (TQ)",
    "Thụy Sỹ": "Thụy Sỹ",
    "Xây-sen": "Xây-Sen",
    "Hà Tĩnh": "Hà Tĩnh",
    "Béc-mu-da": "Béc-mu-da",
    "Cần Thơ": "Cần Thơ",
    "CHLB Đức": "Đức",
    "Béc-mu-đa": "Béc-mu-da",
    "Quảng Nam": "Quảng Nam",
    "Bỉ": "Bỉ",
    "Xây Sen": "Xây-Sen",
    "Ôx-trây-li-a": "Ôx-trây-li-a",
    "Quần đảo Virgin thuộc Mỹ": "Quần đảo Vigin thuộc Anh",
    "Phi-li-pin": "Phi-li-pin",
    "Phi-lip-pin": "Phi-li-pin",
    "Hậu Giang": "Hậu Giang",
    "Quần đảo Cay-man": "Quần đảo Cay-man",
    "Ô-xtrây-li-a": "Ôx-trây-li-a",
    "Quần đảo Cây- men": "Quần đảo Cay-man",
    "Bến Tre": "Bến Tre",
    "Quảng Bình": "Quảng Bình",
    "Bác Giang": "Bắc Giang",
    "Vĩnh Long": "Vĩnh Long",
    "Tiền Giang": "Tiền Giang",
    "Bình Thuận": "Bình Thuận",
    "Hồng Công": "Đặc khu hành chính Hồng Kông (TQ)",
    "Lúc-xăm-bua": "Lúc-xăm-bua",
    "Thừa Thiên - Huế": "Thừa Thiên - Huế",
    "Đặc khu Hành chính Hồng Công (TQ)": "Đặc khu hành chính Hồng Kông (TQ)",
    "CHND Trung Hoa": "Trung Quốc",
    "Niu Di-lân": "Niu Di-lân",
    "Luxembourg": "Lúc-xăm-bua",
    "Anguilla": "Anguilla",
    "Hồng Kông": "Đặc khu hành chính Hồng Kông (TQ)",
    "Phú Yên": "Phú Yên",
    "Ca-na-đa": "Ca-na-da",
    "Quần đảo Cay-men": "Quần đảo Cay-man",
    "Bê-li-xê": "Bê-li-xê",
    "Thừa Thiên - Huế": "Thừa Thiên - Huế",
    "Thanh Hóa": "Thanh Hóa",
    "Vĩnh Long": "Vĩnh Long",
    "Bến Tre": "Bến Tre",
    "Đà Nẵng": "Đà Nẵng",
    "Thái Lan": "Thái Lan",
    "Quần đảo Cay-men": "Quần đảo Cay-man",
    "Đan Mạch": "Đan Mạch",
    "Cộng hòa Síp": "Cộng hòa Síp",
    "Ninh Thuận": "Ninh Thuận",
    "Thái Nguyên": "Thái Nguyên",
    "Bạc Liêu": "Bạc Liêu",
    "Quảng Ngãi": "Quảng Ngãi",
    "Kiên Giang": "Kiên Giang",
    "Vương quốc Bỉ": "Vương quốc Bỉ",
    "Ma-cao": "Ma-cao",
    "In-đô-nê-xi-a": "In-đô-nê-xi-a",
    "Na Uy": "Na-uy",
    "Trà Vinh": "Trà Vinh",
    "Bơ-mu-đa": "Bơ-mu-đa",
    "Thụy Sĩ": "Thụy Sỹ",
    "Các Tiểu vương quốc A-rập Thống nhất": "Các Tiểu vương quốc A-rập Thống nhất",
    "Đắk Lắk": "Đăk Lăk",
    "Pa-ki-xtan": "Pa-ki-xtan",
    "Khánh Hòa": "Khánh Hòa",
    "Ô-man": "Ô-man",
    "Tây Ấn thuộc Anh": "Tây Ấn thuộc Anh",
    "Trà Vinh": "Trà Vinh",
    "Hung-ga-ri": "Hung-ga-ri",
    "Xlô-va-ki": "Xlô-va-ki",
    "Xlô-va-ki-a": "Xlô-va-ki",
    "Đắc Lắk": "Đăk Lăk",
    "Lâm Đồng": "Lâm Đồng",
    "Ni-giê-ri-a": "Ni-giê-ri-a",
    "Bác-ba-đốt": "Bác-ba-đốt",
    "Bru-nây": "Bru-nây",
    "Đặc khu Hành chính Hồng Kông (TQ)": "Đặc khu hành chính Hồng Kông (TQ)",
    "Đặ khu HC Hồng Kông (TQ)": "Đặc khu hành chính Hồng Kông (TQ)",
    "Thổ Nhĩ Kỳ": "Thổ Nhĩ Kỳ",
    "Quần đảo Cây-men": "Quần đảo Cay-man",
}

for pl in dfs_fdi["Phân Loại"].unique().tolist():
    assert pl in convert_pl.keys(), pl


dfs_fdi["Phân Loại"] = dfs_fdi["Phân Loại"].map(convert_pl)

In [32]:
convert_to_mien = {
    "TỔNG SỐ": "TỔNG SỐ",
    "Lâm Đồng": "Tây Nguyên",
    "Đăk Lăk": "Tây Nguyên",
    "Bắc Giang": "Miền Bắc",
    "Bắc Ninh": "Miền Bắc",
    "Hà Nam": "Miền Bắc",
    "Hà Nội": "Miền Bắc",
    "Hưng Yên": "Miền Bắc",
    "Hải Dương": "Miền Bắc",
    "Hải Phòng": "Miền Bắc",
    "Nam Định": "Miền Bắc",
    "Nghệ An": "Miền Bắc",
    "Ninh Bình": "Miền Bắc",
    "Phú Thọ": "Miền Bắc",
    "Phú Yên": "Miền Bắc",
    "Quảng Ninh": "Miền Bắc",
    "Thanh Hóa": "Miền Bắc",
    "Thái Bình": "Miền Bắc",
    "Thái Nguyên": "Miền Bắc",
    "Vĩnh Phúc": "Miền Bắc",
    "Yên Bái": "Miền Bắc",
    "Bến Tre": "Miền Nam",
    "Bà Rịa - Vũng Tàu": "Miền Nam",
    "Bình Dương": "Miền Nam",
    "Bình Phước": "Miền Nam",
    "Bạc Liêu": "Miền Nam",
    "Cần Thơ": "Miền Nam",
    "Hậu Giang": "Miền Nam",
    "Kiên Giang": "Miền Nam",
    "Long An": "Miền Nam",
    "Sóc Trăng": "Miền Nam",
    "TP. Hồ Chí Minh": "Miền Nam",
    "Tiền Giang": "Miền Nam",
    "Trà Vinh": "Miền Nam",
    "Tây Ninh": "Miền Nam",
    "Vĩnh Long": "Miền Nam",
    "Đồng Nai": "Miền Nam",
    "Bình Thuận": "Miền Trung",
    "Hà Tĩnh": "Miền Trung",
    "Khánh Hòa": "Miền Trung",
    "Ninh Thuận": "Miền Trung",
    "Quảng Ngãi": "Miền Trung",
    "Quảng Bình": "Miền Trung",
    "Quảng Nam": "Miền Trung",
    "Thừa Thiên - Huế": "Miền Trung",
    "Đà Nẵng": "Miền Trung",
    "In-đô-nê-xi-a": "Châu Á",
    "Nhật Bản": "Châu Á",
    "Trung Quốc": "Châu Á",
    "Đài Loan": "Châu Á",
    "Đặc khu hành chính Hồng Kông (TQ)": "Châu Á",
    "Ấn Độ": "Châu Á",
    "Hàn Quốc": "Châu Á",
    "Thái Lan": "Châu Á",
    "Phi-li-pin": "Châu Á",
    "Các Tiểu vương quốc A-rập Thống nhất": "Châu Á",
    "Bru-nây": "Châu Á",
    "I-xa-ren": "Châu Á",
    "Ma-cao": "Châu Á",
    "Ma-lai-xi-a": "Châu Á",
    "Pa-ki-xtan": "Châu Á",
    "Ô-man": "Châu Á",
    "Xin-ga-po": "Châu Á",
    "Bỉ": "Châu Âu",
    "Na-uy": "Châu Âu",
    "Pháp": "Châu Âu",
    "Phần Lan": "Châu Âu",
    "Thụy Sỹ": "Châu Âu",
    "Thụy Điển": "Châu Âu",
    "Vương quốc Anh": "Châu Âu",
    "Vương quốc Bỉ": "Châu Âu",
    "Đan Mạch": "Châu Âu",
    "Đức": "Châu Âu",
    "Hà Lan": "Châu Âu",
    "Cộng hòa Síp": "Châu Âu",
    "Hung-ga-ri": "Châu Âu",
    "I-ta-li-a": "Châu Âu",
    "Lúc-xăm-bua": "Châu Âu",
    "Xlô-va-ki": "Châu Âu",
    "Anguilla": "Châu Mỹ",
    "Bác-ba-đốt": "Châu Mỹ",
    "Béc-mu-da": "Châu Mỹ",
    "Bê-li-xê": "Châu Mỹ",
    "Bơ-mu-đa": "Châu Mỹ",
    "Ca-na-da": "Châu Mỹ",
    "Quần đảo Cay-man": "Châu Mỹ",
    "Quần đảo Vigin thuộc Anh": "Châu Mỹ",
    "Tây Ấn thuộc Anh": "Châu Mỹ",
    "Hoa Kỳ": "Châu Mỹ",
    "Ni-giê-ri-a": "Châu Phi",
    "Xây-Sen": "Châu Phi",
    "Niu Di-lân": "Châu Úc",
    "Xa-moa": "Châu Úc",
    "Ôx-trây-li-a": "Châu Úc",
    "Thổ Nhĩ Kỳ": "Châu Âu",
    "Quần đảo Cây-men": "Châu Mỹ",
}


for pl in dfs_fdi["Phân Loại"].unique().tolist():
    assert pl in convert_to_mien.keys(), pl

dfs_fdi["Vùng Miền"] = dfs_fdi["Phân Loại"].map(convert_to_mien)

In [33]:
convert_to_mien = {
    "TỔNG SỐ": "TỔNG SỐ",
    "Lâm Đồng": "Trong Nước",
    "Đăk Lăk": "Trong Nước",
    "Bắc Giang": "Trong Nước",
    "Bắc Ninh": "Trong Nước",
    "Hà Nam": "Trong Nước",
    "Hà Nội": "Trong Nước",
    "Hưng Yên": "Trong Nước",
    "Hải Dương": "Trong Nước",
    "Hải Phòng": "Trong Nước",
    "Nam Định": "Trong Nước",
    "Nghệ An": "Trong Nước",
    "Ninh Bình": "Trong Nước",
    "Phú Thọ": "Trong Nước",
    "Phú Yên": "Trong Nước",
    "Quảng Ninh": "Trong Nước",
    "Thanh Hóa": "Trong Nước",
    "Thái Bình": "Trong Nước",
    "Thái Nguyên": "Trong Nước",
    "Vĩnh Phúc": "Trong Nước",
    "Yên Bái": "Trong Nước",
    "Bến Tre": "Trong Nước",
    "Bà Rịa - Vũng Tàu": "Trong Nước",
    "Bình Dương": "Trong Nước",
    "Bình Phước": "Trong Nước",
    "Bạc Liêu": "Trong Nước",
    "Cần Thơ": "Trong Nước",
    "Hậu Giang": "Trong Nước",
    "Kiên Giang": "Trong Nước",
    "Long An": "Trong Nước",
    "Sóc Trăng": "Trong Nước",
    "TP. Hồ Chí Minh": "Trong Nước",
    "Tiền Giang": "Trong Nước",
    "Trà Vinh": "Trong Nước",
    "Tây Ninh": "Trong Nước",
    "Vĩnh Long": "Trong Nước",
    "Đồng Nai": "Trong Nước",
    "Bình Thuận": "Trong Nước",
    "Hà Tĩnh": "Trong Nước",
    "Khánh Hòa": "Trong Nước",
    "Ninh Thuận": "Trong Nước",
    "Quảng Ngãi": "Trong Nước",
    "Quảng Bình": "Trong Nước",
    "Quảng Nam": "Trong Nước",
    "Thừa Thiên - Huế": "Trong Nước",
    "Đà Nẵng": "Trong Nước",
    "In-đô-nê-xi-a": "Nước Ngoài",
    "Nhật Bản": "Nước Ngoài",
    "Trung Quốc": "Nước Ngoài",
    "Đài Loan": "Nước Ngoài",
    "Đặc khu hành chính Hồng Kông (TQ)": "Nước Ngoài",
    "Ấn Độ": "Nước Ngoài",
    "Hàn Quốc": "Nước Ngoài",
    "Thái Lan": "Nước Ngoài",
    "Phi-li-pin": "Nước Ngoài",
    "Các Tiểu vương quốc A-rập Thống nhất": "Nước Ngoài",
    "Bru-nây": "Nước Ngoài",
    "I-xa-ren": "Nước Ngoài",
    "Ma-cao": "Nước Ngoài",
    "Ma-lai-xi-a": "Nước Ngoài",
    "Pa-ki-xtan": "Nước Ngoài",
    "Ô-man": "Nước Ngoài",
    "Xin-ga-po": "Nước Ngoài",
    "Bỉ": "Nước Ngoài",
    "Na-uy": "Nước Ngoài",
    "Pháp": "Nước Ngoài",
    "Phần Lan": "Nước Ngoài",
    "Thụy Sỹ": "Nước Ngoài",
    "Thụy Điển": "Nước Ngoài",
    "Vương quốc Anh": "Nước Ngoài",
    "Vương quốc Bỉ": "Nước Ngoài",
    "Đan Mạch": "Nước Ngoài",
    "Đức": "Nước Ngoài",
    "Hà Lan": "Nước Ngoài",
    "Cộng hòa Síp": "Nước Ngoài",
    "Hung-ga-ri": "Nước Ngoài",
    "I-ta-li-a": "Nước Ngoài",
    "Lúc-xăm-bua": "Nước Ngoài",
    "Xlô-va-ki": "Nước Ngoài",
    "Anguilla": "Nước Ngoài",
    "Bác-ba-đốt": "Nước Ngoài",
    "Béc-mu-da": "Nước Ngoài",
    "Bê-li-xê": "Nước Ngoài",
    "Bơ-mu-đa": "Nước Ngoài",
    "Ca-na-da": "Nước Ngoài",
    "Quần đảo Cay-man": "Nước Ngoài",
    "Quần đảo Vigin thuộc Anh": "Nước Ngoài",
    "Tây Ấn thuộc Anh": "Nước Ngoài",
    "Hoa Kỳ": "Nước Ngoài",
    "Ni-giê-ri-a": "Nước Ngoài",
    "Xây-Sen": "Nước Ngoài",
    "Niu Di-lân": "Nước Ngoài",
    "Xa-moa": "Nước Ngoài",
    "Ôx-trây-li-a": "Nước Ngoài",
    "Thổ Nhĩ Kỳ": "Châu Âu",
    "Quần đảo Cây-men": "Châu Mỹ",
}


for pl in dfs_fdi["Phân Loại"].unique().tolist():
    assert pl in convert_to_mien.keys(), pl

dfs_fdi["Trong Nước"] = dfs_fdi["Phân Loại"].map(convert_to_mien)

In [34]:
dfs_fdi

,Phân Loại,Số dự án,Vốn đăng ký cấp mới,Vốn đăng ký điều chỉnh,Vốn đăng ký,Date,Month,PreMonth,Số dự án trong tháng,Vốn đăng ký cấp mới trong tháng,Vốn đăng ký điều chỉnh trong tháng,Vốn đăng ký trong tháng,Vùng Miền,Trong Nước
0,TỔNG SỐ,966,7110.212866,1234.628932,8344.841798,2024-04-29,2024-04,2024-03,322,2336.203461,300.029434,2636.232895,TỔNG SỐ,TỔNG SỐ
1,Bà Rịa - Vũng Tàu,17,1521.095655,-9.557457,1511.538198,2024-04-29,2024-04,2024-03,12,1225.679913,-9.557457,1174.716009,Miền Nam,Trong Nước
2,Hà Nội,73,1008.129751,78.998896,1087.128647,2024-04-29,2024-04,2024-03,26,103.883749,57.432063,161.315812,Miền Bắc,Trong Nước
3,Quảng Ninh,15,737.965228,-4.507555,733.457673,2024-04-29,2024-04,2024-03,5,163.229628,2.705882,165.93551,Miền Bắc,Trong Nước
4,Bắc Ninh,142,595.713546,369.239817,964.953363,2024-04-29,2024-04,2024-03,54,114.614023,129.485,244.099023,Miền Bắc,Trong Nước
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3569,Quần đảo Vigin thuộc Anh,3,NaN,NaN,9.4,2016-01-29,2016-01,2015-12,NaN,NaN,NaN,NaN,Châu Mỹ,Nước Ngoài
3570,Đài Loan,6,NaN,NaN,8.7,2016-01-29,2016-01,2015-12,NaN,NaN,NaN,NaN,Châu Á,Nước Ngoài
3571,Thái Lan,3,NaN,NaN,6.3,2016-01-29,2016-01,2015-12,NaN,NaN,NaN,NaN,Châu Á,Nước Ngoài
3572,Hà Lan,1,NaN,NaN,6,2016-01-29,2016-01,2015-12,NaN,NaN,NaN,NaN,Châu Âu,Nước Ngoài


In [35]:
dfs_fdi.to_csv('../csv/fdi.csv')

In [ ]:
nganh = "TỔNG SỐ"

dfs_fdi_hanghoa = dfs_fdi[dfs_fdi["Phân Loại"] == nganh]
dfs_fdi.loc[dfs_fdi["Phân Loại"] == nganh, "Vốn đăng ký so với cùng kỳ"] = dfs_fdi_hanghoa["Vốn đăng ký"].pct_change(-12)
dfs_fdi

/tmp/ipykernel_113586/4230591522.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfs_fdi.loc[dfs_fdi["Phân Loại"] == nganh, "Vốn đăng ký so với cùng kỳ"] = dfs_fdi_hanghoa["Vốn đăng ký"].pct_change(-12)


,Phân Loại,Số dự án,Vốn đăng ký cấp mới,Vốn đăng ký điều chỉnh,Vốn đăng ký,Date,Month,PreMonth,Số dự án trong tháng,Vốn đăng ký cấp mới trong tháng,Vốn đăng ký điều chỉnh trong tháng,Vốn đăng ký trong tháng,Vùng Miền,Trong Nước,Vốn đăng ký so với cùng kỳ
0,TỔNG SỐ,644,4774.009405,934.599498,5708.608902,2024-03-29,2024-03,2024-02,239,1178.071623,492.502551,1670.574174,TỔNG SỐ,TỔNG SỐ,0.134974
1,Hà Nội,47,904.246002,21.566832,925.812834,2024-03-29,2024-03,2024-02,20,34.414297,12.485748,46.900045,Miền Bắc,Trong Nước,NaN
2,Quảng Ninh,10,574.7356,-7.213437,567.522163,2024-03-29,2024-03,2024-02,2,96.4,0.0,96.4,Miền Bắc,Trong Nước,NaN
3,Bắc Ninh,88,481.099523,239.754817,720.85434,2024-03-29,2024-03,2024-02,34,309.699975,137.196431,446.896406,Miền Bắc,Trong Nước,NaN
4,Thái Nguyên,7,470.657,6.81,477.467,2024-03-29,2024-03,2024-02,4,12.8,2.0,14.8,Miền Bắc,Trong Nước,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3529,Quần đảo Vigin thuộc Anh,3,NaN,NaN,9.4,2016-01-29,2016-01,2015-12,NaN,NaN,NaN,NaN,Châu Mỹ,Nước Ngoài,NaN
3530,Đài Loan,6,NaN,NaN,8.7,2016-01-29,2016-01,2015-12,NaN,NaN,NaN,NaN,Châu Á,Nước Ngoài,NaN
3531,Thái Lan,3,NaN,NaN,6.3,2016-01-29,2016-01,2015-12,NaN,NaN,NaN,NaN,Châu Á,Nước Ngoài,NaN
3532,Hà Lan,1,NaN,NaN,6,2016-01-29,2016-01,2015-12,NaN,NaN,NaN,NaN,Châu Âu,Nước Ngoài,NaN


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_fdi[(dfs_fdi["Phân Loại"] == "TỔNG SỐ")]

fig = px.area(df_show, x="Date", y="Vốn đăng ký so với cùng kỳ", color="Phân Loại")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()